# Naive Bayes Klasifikasi

**Nama  : Shohibun Naufal**

**NIM   : 210411100013**

**Kelas : Penambangan Data-C**

Naive Bayes adalah sebuah algoritma pembelajaran mesin yang umum digunakan untuk klasifikasi dan analisis teks. Algoritma ini didasarkan pada teorema Bayes, yang menyatakan bahwa probabilitas suatu kejadian terjadi dapat dihitung berdasarkan probabilitas terkait dengan kejadian tersebut.

Dalam klasifikasi teks, Naive Bayes digunakan untuk menentukan probabilitas bahwa suatu dokumen termasuk ke dalam kategori tertentu (misalnya positif atau negatif) berdasarkan kemunculan kata-kata tertentu dalam dokumen tersebut. Algoritma ini dinamakan "naive" karena asumsi dasar yang digunakan adalah bahwa setiap fitur (kata) dalam dokumen adalah independen satu sama lainnya, meskipun kenyataannya beberapa kata mungkin saling terkait.

1. import library pandas, numpy, dan sklearn yang diperlukan

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

2. import data kita

In [4]:
df = pd.read_csv('https://gist.githubusercontent.com/shohibunnaufal/e07ed74f62ff07d74ce539a602c7a352/raw/b7149502289a717a5bb346aacebfcd4845abbc39/heartdisease.csv')

jumlah_baris = df.shape[0]
df.head(jumlah_baris)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


4. Menangani data hilang atau missing value

> dibawah ini menghitung data kosong:

In [5]:
df_copy = df.copy()
df_copy.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

> ganti nilai yang mengandung tanda tanya (?) menjadi nilai kosong,dan mengecek berapa nilai yang kosong

In [6]:
df_copy = df_copy.replace('?',np.nan)

In [7]:
df_copy.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

5. Cek type data dari setiap fitur

In [8]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


7. Melakukan Normalisasi data mengubah setiap nilai ke dalam rentang 0 hingga 1,dengan metode min-max dan juga metode simple feature scalling(z-score) .namun untuk Kolom target(kolom species ) tak akan di normalisasi

In [9]:
target_column = df_copy['target']
df_copy = df_copy[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']].copy()
df_copy = (df_copy - df_copy.min())/(df_copy.max() - df_copy.min())
df_copy['target'] = target_column.copy()
df_copy.head(jumlah_baris)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.708333,1.0,1.000000,0.481132,0.244292,1.0,0.0,0.603053,0.0,0.370968,0.0,0.00,0.333333,1
1,0.166667,1.0,0.666667,0.339623,0.283105,0.0,0.5,0.885496,0.0,0.564516,0.0,0.00,0.666667,1
2,0.250000,0.0,0.333333,0.339623,0.178082,0.0,0.0,0.770992,0.0,0.225806,1.0,0.00,0.666667,1
3,0.562500,1.0,0.333333,0.245283,0.251142,0.0,0.5,0.816794,0.0,0.129032,1.0,0.00,0.666667,1
4,0.583333,0.0,0.000000,0.245283,0.520548,0.0,0.5,0.702290,1.0,0.096774,1.0,0.00,0.666667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.583333,0.0,0.000000,0.433962,0.262557,0.0,0.5,0.396947,1.0,0.032258,0.5,0.00,1.000000,0
299,0.333333,1.0,1.000000,0.150943,0.315068,0.0,0.5,0.465649,0.0,0.193548,0.5,0.00,1.000000,0
300,0.812500,1.0,0.000000,0.471698,0.152968,1.0,0.5,0.534351,0.0,0.548387,0.5,0.50,1.000000,0
301,0.583333,1.0,0.000000,0.339623,0.011416,0.0,0.5,0.335878,1.0,0.193548,0.5,0.25,1.000000,0


8. Pemotongan Data atau Data Slicing

Untuk memisahkan data menjadi data training dan data test, kita dapat dengan mudah menggunakan metode train_test_split() dari sklearn.

Setelah memanggil metode ini dengan argumen input berupa data fitur dan target, variabel features_train akan berisi data fitur untuk training, variabel features_test akan berisi data fitur untuk test, variabel target_train akan berisi target untuk training, dan variabel target_test akan berisi target untuk test.

Dengan cara ini, kita dapat memisahkan dataset kita secara acak menjadi data training dan data test untuk melatih dan menguji model kita secara efektif.

In [13]:
features = df_copy.values[:,:13]  #Variabel features diisi dengan nilai features dari dataset adult_df_rev dengan slicing [:,:13] artinya mengambil seluruh baris dan kolom 0 hingga kolom 3 (kolom ke-13 tidak diambil karena merupakan variabel target).
target = df_copy.values[:,13]  #Variabel target diisi dengan nilai target dari dataset adult_df_rev dengan slicing [:,13] artinya mengambil seluruh baris dan hanya kolom ke-13 (variabel target).
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size = 0.33, random_state = 10)  #Kemudian menggunakan fungsi train_test_split, dataset yang sudah dipisahkan dijadikan argumen, dengan tambahan argumen test_size = 0.33 yang berarti dataset akan dibagi menjadi 33% data uji dan 67% data latih, dan random_state = 10 sebagai seed untuk menghasilkan hasil yang sama setiap kali dijalankan.

9. Implementasi Gaussian Naive Bayes

> Setelah selesai preprocessing data. saatnya menerapkan algoritma pembelajaran mesin atau machine learning. Kita akan menggunakan modul GaussianNB sklearn:

In [14]:
clf = GaussianNB()
clf.fit(features_train, target_train)
target_pred = clf.predict(features_test)

12. Akurasi model Gaussian Naive Bayes

In [15]:
accuracy = accuracy_score(target_test, target_pred, normalize = True)

print(f"Akurasi model KNN: {accuracy:.2%}")

Akurasi model KNN: 79.00%


## Kesimpulan

Berdasarkan nilai akurasi sebesar 79.00% yang diperoleh dari model yang telah dilatih, dapat dikatakan bahwa data kita sudah cukup bagus. Namun, untuk memastikan kinerja model Gaussian Naive Bayes yang telah terlatih, kita dapat membuat kumpulan data uji acak dan menguji model tersebut untuk melihat seberapa baik performanya. Dengan cara ini, kita dapat mengevaluasi apakah model yang telah terlatih pada data training dapat diterapkan pada data yang belum pernah dilihat sebelumnya dengan baik atau tidak.